In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [52]:
class1_ls = pd.read_csv("Group15/LS_Group15/Class1.txt", sep=' ', header=None)
class2_ls = pd.read_csv("Group15/LS_Group15/Class2.txt", sep=' ', header=None)
class3_ls = pd.read_csv("Group15/LS_Group15/Class3.txt", sep=' ', header=None)

In [53]:
class1_ls.head()

,0,1
0,-2.2365,2.3224
1,2.4445,4.5807
2,1.8720,1.1315
3,5.1338,1.0316
4,2.1731,2.2868


In [54]:
class1_ls = class1_ls.assign(Class=1)
class2_ls = class2_ls.assign(Class=2)
class3_ls = class3_ls.assign(Class=3)

In [55]:
class1_ls.head()

,0,1,Class
0,-2.2365,2.3224,1
1,2.4445,4.5807,1
2,1.8720,1.1315,1
3,5.1338,1.0316,1
4,2.1731,2.2868,1


In [56]:
df = pd.concat([class1_ls, class2_ls, class3_ls], ignore_index=True)
df

,0,1,Class
0,-2.2365,2.3224,1
1,2.4445,4.5807,1
2,1.8720,1.1315,1
3,5.1338,1.0316,1
4,2.1731,2.2868,1
...,...,...,...
1495,17.1120,-6.9613,3
1496,16.8170,-2.7400,3
1497,14.7910,-3.5821,3
1498,17.1720,-9.3333,3


In [57]:
from sklearn.model_selection import train_test_split

X, y = df.drop('Class', axis=1), df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

In [58]:
X_train.head()

,0,1
916,5.2645,-13.1920
728,4.7062,-10.5740
569,8.9554,-16.9660
814,3.3651,-8.0589
1217,15.9320,-6.3017


In [59]:
df_train = pd.concat([X_train, y_train], axis=1)
df_train.head()

,0,1,Class
916,5.2645,-13.1920,2
728,4.7062,-10.5740,2
569,8.9554,-16.9660,2
814,3.3651,-8.0589,2
1217,15.9320,-6.3017,3


In [60]:
df_train = df_train.reset_index(drop=True)
df_train.head()

,0,1,Class
0,5.2645,-13.1920,2
1,4.7062,-10.5740,2
2,8.9554,-16.9660,2
3,3.3651,-8.0589,2
4,15.9320,-6.3017,3


In [61]:
df_test = pd.concat([X_test, y_test], axis=1)
df_test.head()

,0,1,Class
1057,13.7200,-10.44500,3
775,6.2552,-11.98100,2
900,4.9949,-9.98170,2
1003,15.8790,-4.78780,3
100,3.2954,-0.40392,1


In [62]:
df_test = df_test.reset_index(drop=True)
df_test.head()

,0,1,Class
0,13.7200,-10.44500,3
1,6.2552,-11.98100,2
2,4.9949,-9.98170,2
3,15.8790,-4.78780,3
4,3.2954,-0.40392,1


# Bayes Classifier 1:
Covariance matrix for all the classes is the same and is $\sigma^2 I$ 

In [63]:
class BayesClassifier1:
    def __init__(self):
        pass

    def __find_rep_temp(self, df_train):
        class_variances = df_train.groupby('Class').var()
        avg_var = class_variances.mean(axis=0)

        self.M = df_train['Class'].nunique()

        self.cov_mat = np.zeros((len(avg_var), len(avg_var)))
        for i in range(len(avg_var)):
            self.cov_mat[i][i] = avg_var[i]
        
        self.means = df_train.groupby('Class').mean()

        self.priors = df_train['Class'].value_counts() / len(df_train)

    def fit(self, X_train, y_train):
        df_train = pd.concat([X_train, y_train], axis=1)
        df_train = df_train.reset_index(drop=True)
    
        # finds the sigma and the mus
        self.__find_rep_temp(df_train)

    def __discriminant(self, x, mu, prior):
        return -1/2 * (np.transpose(x - mu) @ np.linalg.inv(self.cov_mat) @ (x - mu)) + np.log(prior)

    def predict(self, X_test):
        y_pred = []

        for idx in range(len(X_test)):
            x = X_test.iloc[idx]
            gs = []

            for i in range(self.M):
                gs.append(self.__discriminant(x, self.means.iloc[i], self.priors.iloc[i]))
        
            y_pred.append(np.argmax(gs) + 1)

        return y_pred


In [64]:
def confusion_mat(y_pred, y_test):
    M = np.unique(y_test).size

    conf_mat = np.zeros((M, M))

    for i in range(len(y_test)):
        conf_mat[y_pred[i]-1][y_test[i]-1] += 1
    
    return conf_mat

In [65]:
bc1 = BayesClassifier1()
bc1.fit(X_train, y_train)
y_pred = bc1.predict(X_test)
confusion_mat(y_pred, np.array(y_test))

array([[146.,   0.,   0.],
       [  0., 161.,   1.],
       [  0.,   0., 142.]])